In [1]:
import ee
import geemap
from tqdm import tqdm
from osgeo import gdal, ogr
ee.Initialize()

In [2]:
# Map = geemap.Map(center=(40, -100), zoom=4)
# Map

In [3]:
def bitwiseExtract(value, fromBit):
    maskSize = ee.Number(1).add(fromBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


def cloudfree_landsat(image):
    qa = image.select('pixel_qa')
    cloudState = bitwiseExtract(qa, 5) 
    cloudShadowState = bitwiseExtract(qa, 3)
    mask = cloudState.eq(0).And(cloudShadowState.eq(0))
    return image.updateMask(mask)


def get_band_1(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B2')
    if year < 2013:
        image = image.select('B1')
    return image.clip(shp)

def get_band_2(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B3')
    if year < 2013:
        image = image.select('B2')
    return image.clip(shp)

def get_band_3(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B4')
    if year < 2013:
        image = image.select('B3')
    return image.clip(shp)

def get_band_4(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B5')
    if year < 2013:
        image = image.select('B4')
#         print(image.getInfo(), year, 'NDVI')
    return image.clip(shp)

def get_band_5(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B6')
    if year < 2013:
        image = image.select('B5')
    return image.clip(shp)

def get_band_6(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B7')
    if year < 2013:
        image = image.select('B6')
    return image.clip(shp)

def get_band_7(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    image = image.select('B7')
    return image.clip(shp)


def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def addNDVI75(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)


def addNDMI(image):
    ndmi = image.normalizedDifference(['B5', 'B6']).rename('NDMI')
    return image.addBands(ndmi)


def addNDMI75(image):
    ndmi = image.normalizedDifference(['B4', 'B5']).rename('NDMI')
    return image.addBands(ndmi)


def addNBR(image):
    nbr = image.normalizedDifference(['B5', 'B7']).rename('NBR')
    return image.addBands(nbr)


def addNBR75(image):
    nbr = image.normalizedDifference(['B4', 'B7']).rename('NBR')
    return image.addBands(nbr)
    
 
def get_ndvi(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNDVI(image).select('NDVI')
    if year < 2013:
        image = addNDVI75(image).select('NDVI')
    return image

    
def get_ndmi(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNDMI(image).select('NDMI')
    if year < 2013:
        image = addNDMI75(image).select('NDMI')
    return image

    
    
def get_nbr(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNBR(image).select('NBR')
    if year < 2013:
        image = addNBR75(image).select('NBR')
    return image



def add_texture(nbr):
    square = ee.Kernel.square(**{'radius': 5})
    return nbr.multiply(1000).toInt().glcmTexture(**{
    'kernel': square,
    'size': 5,
    })


def feature_extract(shp, image, name):
    return image.reduceRegions(**{
    'collection': shp,
    'reducer': ee.Reducer.mean().setOutputs(name),
    'scale': 90
    })


def get_index(ls, shp, year):
    shp_feature = feature_extract(shp, get_ndvi(ls, shp, year).select('NDVI'), ['NDVI'])
    shp_feature = feature_extract(shp_feature, get_ndmi(ls, shp, year).select('NDMI'), ['NDMI'])
    shp_feature = feature_extract(shp_feature, get_nbr(ls, shp, year).select('NBR'), ['NBR'])
    shp_feature = feature_extract(shp_feature, get_band_1(ls, shp, year), ['B1'])
    shp_feature = feature_extract(shp_feature, get_band_2(ls, shp, year), ['B2'])
    shp_feature = feature_extract(shp_feature, get_band_3(ls, shp, year), ['B3'])
    shp_feature = feature_extract(shp_feature, get_band_4(ls, shp, year), ['B4'])
    shp_feature = feature_extract(shp_feature, get_band_5(ls, shp, year), ['B5'])
    shp_feature = feature_extract(shp_feature, get_band_6(ls, shp, year), ['B6'])
    shp_feature = feature_extract(shp_feature, get_band_7(ls, shp, year), ['B7'])
    return shp_feature


def select_ls(year):
    if year >=2013:
        ls = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    elif year >= 2003:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")
    elif year >= 1999:
        ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").merge(ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA"))
    else:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    return ls


def process(shp, year):
    shp_feature = get_index(ee.ImageCollection("LANDSAT/LT05/C01/T1_SR"), shp, year)
    if year != end_year:
        if year == 2011:
            texture = add_texture(get_nbr(select_ls(year + 2), shp, year + 2))
        else:
            texture = add_texture(get_nbr(select_ls(year + 1), shp, year + 1))
        shp_feature = feature_extract(shp_feature, texture.select('NBR_asm'), ['TXU_asm'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_idm'), ['TXU_HOM'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_contrast'), ['TXU_CON'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_sent'), ['TXU_ENT'])
    return shp_feature

In [4]:
# path = "D:\\data\\GEE\\prediction2\\merge\\all_merge.shp"
# driver = ogr.GetDriverByName('ESRI Shapefile')
# shp = driver.Open(path)
# layer = shp.GetLayer()

In [5]:
# data_list = []
# for i in range(30):
#     data_list.append(ee.FeatureCollection([]))
# for i in tqdm(range(len(layer))):
#     feature = layer.GetFeature(i)
#     if feature['type_ne'] == 1 and feature['count'] > 10:
#         year = int(feature['path'][32:36])
#         feature_geo = feature.GetGeometryRef()
#         roi_str = feature_geo.ExportToIsoWkt()
#         roi = roi_str.split('(')[2:]
#         for j in range(len(roi)):
#             roi[j] = roi[j].strip("'").strip(',').strip(')').split(',')
#             for coor in range(len(roi[j])):
#                 roi[j][coor] = [float(roi[j][coor].split(' ')[0]), float(roi[j][coor].split(' ')[1])]
#         for dur_year in range(year + 1, 2021):
#             if dur_year != 2012:
#                 data_list[dur_year - 1991] = data_list[dur_year - 1991].merge(ee.FeatureCollection(ee.Geometry.Polygon(roi)))

In [6]:
taskParams = {
'driveFolder': 'CB_Target_data',
'fileFormat': 'SHP'
}
# for FC in data_list:
FC = ee.FeatureCollection("users/1901818/type-2")
end_year = 2020
year = 1986
task = ee.batch.Export.table(process(FC, year), str(year), taskParams)
task.start()

In [7]:
# Map.addLayer(data_list[0], {}, "test")
# Map.centerObject(data_list[0], 8)